In [5]:
from sklearn import cross_validation
from sklearn.metrics import log_loss,accuracy_score
import numpy as np
import pandas as pd
import random
import json
import hashlib 

In [17]:
def blend_proba(clf,X_train,y,X_test,nfolds=5,save_preds="",save_test_only="",seed=300373,
               save_params="",clf_name="XX",generalizers_params=[],minimal_loss=0,return_score=False,minmizer='log_loss'):
    print("\nBlending with classifier:\n\t%s"%(clf))
    folds = list(cross_validation.StratifiedKFlod(y,nfolds,shuffle=True,random_state=seed))
    print(X_train.shape)
    dataset_blend_train = np.zeros((X_train.shape[0],np.unique(y).shape[0]))
    loss = 0
    for i ,(train_index,test_index) in enumerate(folds) :
        print("Train Fold %s/%s"%(i+1,nfolds))
        fold_X_train = X_train[train_index]
        fold_y_train = y[train_index]
        fold_X_test = y[test_index]
        fold_y_test = y[test_index]
        clf.fit(fold_X_train,fold_y_train)
        fold_preds = clf.predict_proba(fold_X_test)
        print("Logistic loss: %s"%log_loss(fold_y_test,fold_preds))
        #pred to train
        dataset_blend_train[test_index] = fold_preds
        if minimizer == "log_loss":
            loss += log_loss(fold_y_test,fold_preds)
        if minimizer == "accuracy":
            fold_preds_a = np.argmax(fold_preds,axis=1)
            #return the class
            loss += accuracy_score(fold_y_test,fold_preds_a)
        if minimal_loss > 0 and loss > minimal_loss and i ==0:
            return False,False
        fold_preds = np.argmax(fold_preds,axis=1)
        print("Accuracy:    %s"%accuracy_score(fold_y_test,fold_preds))
    avg_loss = loss/float(i+1)
    print("\nAverage:\t%s\n"%avg_loss)
    print("Test Fold 1/1")
    clf.fit(X_train,y)
    dataset_blend_test = clf.predict_proba(X_test)
    
    if clf_name == "XX":
        clf_name = str(clf)[1:3]
    

In [23]:
from __future__ import division
import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale

In [26]:
def logloss(attempt,actual,epsilon=1.0e-15):
    attempt = np.clip(attempt,epsilon,1.0-epsilon)
    return -np.mean(actual*np.log(attempt)+(1.0-actual)*np.log(1.0-attempt))
def blend(clfs,X,y,X_submission):
    np.random.seed(0)
    n_folds = 10
    verbose = True
    shuffle = False
    #xi pai
    if shuffle:
        idx = np.random.permutation(y.size)
        X = X[idx]
        y = y[idx]
    skf = list(StratifiedKFold(y,n_folds))
    print("Creating train and test sets for blending")
    dataset_blend_train = np.zeros((X.shape[0],len(clfs)))
    dataset_blend_test = np.zeros((X_submission.shape[0],len(clfs)))
    for j,clf in enumerate(clfs):
        dataset_blend_test_j = np.zeros((X_submission.shape[0],len(skf)))
        for i,(train,test) in enumerate(skf):
            X_train = X[train]
            y_train = y[train]
            X_test = X[test]
            y_test = y[test]
            clf.fit(X_train,y_train)
            y_submission = clf.predict_proba(X_test)[:,1]
            dataset_blend_train[test,j] = y_submission
            dataset_blend_test_j = clf.predict_proba(X_submission)[:,1]
        dataset_blend_test[:,j] = dataset_blend_test_j.mean(1)
    clf = LogisticRegression()
    clf.fit(dataset_blend_train,y)
    y_submission = clf.predict_proba(dataset_blend_test)[:,1]
    #Linear stretch of predictions to [0,1]
    y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())
    #SAVING 
    return y_submission
    